# A Notebook to iterate through a set of training CSV's and train a SCL ReID model, evaluate model, and save results to a line in a csv

In [5]:
# ---- Necessary Imports

import yaml
import os
from glob import glob
import os
import pandas as pd
import json
import pickle
import numpy as np
import sys

## Correct filepaths in csvs (if needed)


# Iterate through training files

### Specify Training and Evaluation Files 
This method iterates through a folder of training csvs and evaluates them on the same test (query) file using the same reference file each time. 

In [5]:
#dir containing training csvs to start working with 
dir = '/home/gsantiago/summer_bee_data/closed_sets_4_ids_all_colors_once_batch2'

test_file = '/home/gsantiago/summer_bee_data/closed_test_batch1/summer_bee_dataset_closed_test_bee_sample_num_None.csv'
#test on all of batch 2 (open set) gonna also test on all of batch 1 

valid_file = '/home/gsantiago/summer_bee_data/closed_sets_max_ids_batch1/summer_bee_dataset_closed_train_bee_balanced_sample_num_2.csv'
# run valid on smaller subset of test_set to speed training 

#reference knn on smaller subset of batch 1
reference_file = '/home/gsantiago/summer_bee_data/closed_sets_max_ids_batch1/summer_bee_dataset_closed_train_bee_balanced_sample_num_2.csv'


results_file = '../yml_and_csv_files/Few_shot_expirament_results_tracking.csv'

file_paths = []

#### Get list of training files

In [5]:
# Get file list 
for root, dirs, files in os.walk(dir):
    files = files
for f in files:
    print(root+r'/'+f)
    train_file = root+r'/'+f
    file_paths.append(train_file)
#     continue

#files.sort()

/home/gsantiago/summer_bee_data/closed_sets_4_ids_all_colors_once_batch2/summer_bee_dataset_closed_train_bee_4_ids_batch2_sample_num_4.csv
/home/gsantiago/summer_bee_data/closed_sets_4_ids_all_colors_once_batch2/summer_bee_dataset_closed_train_bee_4_ids_batch2_sample_num_8.csv
/home/gsantiago/summer_bee_data/closed_sets_4_ids_all_colors_once_batch2/summer_bee_dataset_closed_train_bee_4_ids_batch2_sample_num_16.csv
/home/gsantiago/summer_bee_data/closed_sets_4_ids_all_colors_once_batch2/summer_bee_dataset_closed_train_bee_4_ids_batch2_sample_num_20.csv
/home/gsantiago/summer_bee_data/closed_sets_4_ids_all_colors_once_batch2/summer_bee_dataset_closed_train_bee_4_ids_batch2_sample_num_max.csv
/home/gsantiago/summer_bee_data/closed_sets_4_ids_all_colors_once_batch2/summer_bee_dataset_closed_train_bee_4_ids_batch2_sample_num_2.csv


In [ ]:
#IF setting two folders to train at once: 

# dir2 = '/home/gsantiago/summer_bee_data/closed_sets_8_ids_monocolor_batch2'
# files2 = []
# for root2, dirs2, files2 in os.walk(dir2):
#     files2 = files2
# for f2 in files2:
#     #print(root+r'/'+f2)
#     train_file2 = root2+r'/'+f2
#     file_paths.append(train_file2)
# #     continue

# print(file_paths)

In [6]:
sys.path.insert(0,"../")

## Loop through files, train and eval model

Code may need to be modified for differnt training sets such as
* run_str parsing to include important variables such as num_ids
* the path to your template yml config file
*

In [6]:
file_paths = ['/home/lmeyers/summer_bee_data_reextract/new_open_max_ids_batch1/summer_bee_dataset_open_train_bee_64_ids_batch1_sample_num_max.csv']#,'/home/lmeyers/summer_bee_data_reextract/new_open_max_ids_batch2/summer_bee_dataset_open_train_bee_64_ids_batch2_sample_num_max.csv']

In [7]:
df = pd.read_csv(file_paths[0])
print(len(df))

10264


In [8]:
# If testing multiple lengths of training, add multiple #'s to epochs_to_test
epochs_to_test = [200]

#Loops through number of epochs
for n in epochs_to_test: 
    
    #Loops through files
    for i in range(len(file_paths)):
        train_file = file_paths[i] #train file path
       
        if i == 0: #for training on different batches at the same time\
            reference_file = '/home/lmeyers/summer_bee_data_reextract/new_open_max_ids_batch2/summer_bee_dataset_open_train_bee_64_ids_batch2_sample_num_02.csv'
            test_file = '/home/lmeyers/summer_bee_data_reextract/new_open_max_ids_batch2/summer_bee_dataset_open_train_bee_64_ids_batch2_sample_num_max.csv'
        else:
            reference_file = '/home/lmeyers/summer_bee_data_reextract/new_open_max_ids_batch1/summer_bee_dataset_open_train_bee_64_ids_batch1_sample_num_02.csv'
            test_file = '/home/lmeyers/summer_bee_data_reextract/new_open_max_ids_batch1/summer_bee_dataset_open_train_bee_64_ids_batch1_sample_num_max.csv'


        ##----------------Filter particular runs (if needed)----------
        #if num_images == 'max':

        ##---------- Initilize new config .yml for new training file---------------
        
        #open config yaml to update experiment params
        with open('../yml_and_csv_files/reid_template.yml', 'r') as fo:
            config = yaml.safe_load(fo)
        
        #------------ Initilize and make new dir for each training set-----

        # Saves wandb folder, checkpoints, and outputs to folder named based on run string, which should contain attrs that differntiate run
        run_str = os.path.basename(train_file)[34:-4] #MAY NEED TO MODIFY BASED ON CSV NAME
        run_dir_name = os.path.join(config['train_settings']['wandb_dir_path'],run_str+'/')
        if not os.path.exists(run_dir_name):
            os.mkdir(run_dir_name)
        split_parts = run_str.rsplit('_', 1) #String parse csv name
        if len(split_parts) > 1: # Check if there is at least one underscore in the string
            # Get the substring after the last underscore
            num_images = split_parts[1]
            num_ids = split_parts[0][:2]
        else:
            # Handle the case where there are no underscores in the string
            num_images = run_str 
        
        # Saves wandb folder, checkpoints, and outputs to folder named based on run string, which should contain attrs that differntiate run
        wandb_name =  config['train_settings']['wandb_project_name']+run_str #set wandb_dir_name
        
        #Update params
        config['model_settings']['num_labels']= num_ids
        print('Num labels ',num_ids)

        #Check if batch size needs to be updated
        df = pd.read_csv(train_file)
        if config['data_settings']['batch_size'] > len(df):
            config['data_settings']['batch_size'] = len(df)
            print('Updated batch to contain all Data. Size = ',len(df))
        
        #Check if print_k needs to be updated for small dataset
        print_k = config['train_settings']['print_k']
        if print_k > len(df)/config['data_settings']['batch_size']:
            print_k = len(df)/config['data_settings']['batch_size']
            config['train_settings']['print_k'] = print_k
            print('Updating print_k to contain whole epoch. Num_batches =',print_k)
        
        #Testing a differnt num of epochs based on loop
        config['train_settings']['num_epochs'] = n

        #updating datafiles
        config['data_settings']['datafiles']['train']=train_file
        config['data_settings']['datafiles']['reference']= reference_file

        #config['data_settings']['datafiles']['train']=train_csv
        config['data_settings']['datafiles']['test'] = test_file
        config['data_settings']['datafiles']['valid']= ''
        config['data_settings']['datafiles']['query']= test_file

        #update Model path
        config['model_settings']['model_path'] = os.path.join(run_dir_name,run_str+'.pth')

        #update pickle_file to prevent being overwritten
        pickle_file = os.path.join(run_dir_name,'results.pkl')
        config['eval_settings']['pickle_file'] = pickle_file
        csv_file = os.path.join(run_dir_name,"results.csv")
        config['eval_settings']['results_file'] = csv_file

        #update wandb_project_name
        config['train_settings']['wandb_project_name'] = wandb_name
        config['train_settings']['wandb_dir_path'] = run_dir_name #this should make a seperate wandb folder for runs

        #save yml
        new_yml_file = run_dir_name+run_str+'.yml'
        with open(new_yml_file, 'w') as fo:
                yaml.dump(config,fo)   

        #---------- actually run training too--------------
        !python pytorch_finetune_and_eval_reid.py --config_file {new_yml_file}

        # Save model to wandb file location to prevent overwriting
        # new dir in wandb/ will be generated each training run
        !cp {config['model_settings']['model_path']} {config['train_settings']['wandb_dir_path']+'/wandb/latest-run/files/'+os.path.basename(config['model_settings']['model_path'])}

        # read python results from pickle file, 
        with open(config['eval_settings']['pickle_file'],'rb') as fi:
            results = pickle.load(fi)  
            
        # Write out run summary to results tracking document
       
        results_dict =  {'run_str': run_str,
                                            'wandb_id':results['wandb_id'],
                                            'num_ids':num_ids,
                                            'num_images_per_id':num_images,
                                            'total_training_images':len(pd.read_csv(train_file))-(len(pd.read_csv(train_file))*config['data_settings']['percent_valid']),
                                            'batch_size':config['data_settings']['batch_size'],
                                            'num_epochs':config['train_settings']['num_epochs'],
                                            'train_loss':results['train_loss'],
                                            'valid_loss':results['valid_loss'],
                                            '1NN':results['1NN_acc'],
                                            '3NN':results['3NN_acc'],
                                            'training_file':train_file,
                                            'reference_file':reference_file,
                                            'query_file':test_file,
                                            'start_time':results['start_time'],
                                            'train_time':results['train_time'],
                                            'unfreeze_epochs': results['unfreeze_epoch'],
                                            'stop_epoch':results['stop_epoch']}
        
        if not os.path.exists(config['eval_settings']['results_file']):
            !touch {config['eval_settings']['results_file']}
            results_df = pd.DataFrame(results_dict,index=[0])
            results_df.to_csv(config['eval_settings']['results_file'])
        else:
            #read df and append row
            results_df = pd.read_csv(config['eval_settings']['results_file'])
            results_df.loc[len(results_df)] = results_dict
            results_df.to_csv(config['eval_settings']['results_file'],index=False)


Num labels  64
2024-10-23 11:17:03.349364: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-10-23 11:17:04.221365: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
/home/lmeyers/anaconda3/envs/mlenv/lib/python3.9/site-packages/transformers/models/clip/feature_extraction_clip.py:28: FutureWarning: The class CLIPFeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use CLIPImageProcessor instead.
  warnings.warn(
finished imports
beginning execution
wandb: Currently logged in as: lqmeyers (meyers_luke_lab). Use `wandb login --relogin` to force relogin
wandb: wandb version 0.18.5 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade
wandb: Tracking ru

## If folder contains csvs from both batches: run below code

In [4]:
# if you want only train on batch1, add the string after the asterisk
#here im training on 32 ids batch1
directory_of_csvs ='/home/gsantiago/summer_bee_data/open_sets/new*04*batch1'

sample_num_of_interest = "sample_num_"

#this variable is for creating a folder that will contain the files we want
type_of_train_wanted = '/home/gsantiago/ReID_model_training/new_auto_train_eval/models_trained/'
#type_of_train_wanted = '/home/lmeyers/ReID_complete/few_shot_experiments/new_training_models/'

#wandb_dir_name is the directory where eveything wandb saves or whatevah
wandb_dir_name = "/home/gsantiago/ReID_model_training/new_auto_train_eval/models_trained/"

results_pickle="/home/gsantiago/ReID_model_training/new_auto_train_eval/"

csv_for_results = "/home/gsantiago/ReID_model_training/new_auto_train_eval/Few_shot_expirament_results_tracking.csv"
epochs_to_test = [1500]
label_col = 'reID'
gpu_id = '1'



## optional!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
## REMOVE LATER


non_trained_models=[

# '/home/gsantiago/summer_bee_data/open_sets/new_open_04_ids_all_colors_batch1/summer_bee_dataset_open_train_bee_4_ids_batch1_sample_num_16.csv',

'/home/gsantiago/summer_bee_data/open_sets/new_open_04_ids_all_colors_batch1/summer_bee_dataset_open_train_bee_4_ids_batch1_sample_num_32.csv',

'/home/gsantiago/summer_bee_data/open_sets/new_open_04_ids_all_colors_batch1/summer_bee_dataset_open_train_bee_4_ids_batch1_sample_num_64.csv'

]
if not os.path.exists(type_of_train_wanted):
    os.mkdir(type_of_train_wanted)



for n in epochs_to_test:
    #loops through every file that starts with that string
    for split in glob(directory_of_csvs):
        #print(split)
        
        #get number of batch to reference and test on the other
        
        batch = split.split('_')[-1]
        #print(batch)
        #print(os.path.dirname(split))
        parent_dir = os.path.dirname(split)
        
        if batch == "batch1":
            query_file = parent_dir +'/open_reference_query_testing_batch2/summer_bee_dataset_closed_test_bee_query_64_ids_batch2.csv'
            reference_file = parent_dir +'/open_reference_query_testing_batch2/summer_bee_dataset_closed_test_bee_reference_64_ids_batch2.csv'
            valid_file = parent_dir +'/open_reference_query_testing_batch2/summer_bee_dataset_closed_test_bee_valid_64_ids_batch2.csv'
        else:
            query_file = parent_dir +'/open_reference_query_testing_batch1/summer_bee_dataset_closed_test_bee_query_64_ids_batch1.csv'
            reference_file = parent_dir +'/open_reference_query_testing_batch1/summer_bee_dataset_closed_test_bee_reference_64_ids_batch1.csv'
            valid_file = parent_dir +'/open_reference_query_testing_batch1/summer_bee_dataset_closed_test_bee_valid_64_ids_batch1.csv'
        split +='/*'+sample_num_of_interest+'*'
        
        
        
        

        #this loops through each directory and takes the csv with the sample num of interest
        # if you want to loop through every sample num, sample_num should be "sample_num"
        for csv in glob(split):
            if csv not in non_trained_models:
                print(csv)
                continue
            
            
            

            train_file = csv
            wandb_name = os.path.basename(os.path.dirname(train_file))
            run_str = os.path.basename(train_file)[:-4]
            run_dir_name = type_of_train_wanted+run_str+'/'
            #print(wandb_name)
            if not os.path.exists(run_dir_name):
                os.mkdir(run_dir_name)
            #print(run_dir_name)




            split_parts = train_file.split('/')[-1].split('_')
            #print(split_parts)
            # Check if there is at least one underscore in the string
            if len(split_parts) > 1:
                # Get the substring after the last underscore
                num_images = split_parts[-1].replace('.csv', '')
                num_ids = split_parts[6]
            else:
                # Handle the case where there are no underscores in the string
                num_images = run_str

            print(num_images, num_ids)

             #open config yaml to update experiment params
            with open('./reid_template.yml', 'r') as fo:
                config = yaml.safe_load(fo)

            #Update params

            #num labels should be taken from pandas because batch1!=batch2

            df = pd.read_csv(train_file)
            num_labels = df['ID'].nunique()

            config['model_settings']['num_labels']= num_labels
            
            
            #ubdating label_col
            config['data_settings']['label_col']=  label_col
            
            #gpu
            config['train_settings']['gpu'] = gpu_id
            print('Num labels ',num_labels)

            #Check if batch size needs to be updated

            config['data_settings']['batch_size'] = 64
            if config['data_settings']['batch_size'] > len(df):
                config['data_settings']['batch_size'] = len(df)
                print('Updated batch to contain all Data. Size = ',len(df))

            #Check if print_k needs to be updated for small dataset
            print_k = config['train_settings']['print_k']
            if print_k > len(df)/config['data_settings']['batch_size']:
                print_k = len(df)/config['data_settings']['batch_size']
                config['train_settings']['print_k'] = print_k
                print('Updating print_k to contain whole epoch. Num_batches =',print_k)

            #Testing a differnt num epochs (EXPIRAMENT HERE)
            config['train_settings']['num_epochs'] = n

            #updating datafiles
            config['data_settings']['datafiles']['train']=train_file
            config['data_settings']['datafiles']['reference']= reference_file


            #config['data_settings']['datafiles']['train']=train_csv
            config['data_settings']['datafiles']['test'] = query_file
            config['data_settings']['datafiles']['valid']= valid_file 
            config['data_settings']['datafiles']['query']= query_file

            #update Model path
            config['model_settings']['model_path'] = run_dir_name+run_str+'.pth'

            #update wandb_project_name
            config['train_settings']['wandb_project_name'] = wandb_name
            config['train_settings']['wandb_dir_path'] = wandb_dir_name + run_str #this should make a seperate wandb folder for runs
            
            #pickle_config_file
            config['eval_settings']['pickle_file'] = results_pickle+ 'results.pkl'
            config['eval_settings']['results_file']= csv_for_results
            
#             #"""
#             #skipping max because I already ran it
#             if num_images not in [64, '64'] and 'monocolor' not in run_str:
#                 print('Skipping')
#                 continue
#             #"""

            #save yml
            new_yml_file = run_dir_name+run_str+'.yml'
            with open(new_yml_file, 'w') as fo:
                    yaml.dump(config,fo)   

            #---------- actually run training too--------------
            !python3 pytorch_train_and_eval_reid_2.py --config_file {new_yml_file}

            # Save model to wandb file location to prevent overwriting
            !cp {config['model_settings']['model_path']} {config['train_settings']['wandb_dir_path']+'/wandb/latest-run/files/'+os.path.basename(config['model_settings']['model_path'])}

            with open(config['eval_settings']['pickle_file'],'rb') as fi:
                results = pickle.load(fi)  

            # Write out run summary to results tracking document
            results_df = pd.read_csv(config['eval_settings']['results_file'])
            results_df.loc[len(results_df)] = {'run_str': run_str,
                                                'wandb_id':results['wandb_id'],
                                                'num_ids':num_ids,
                                                'num_images_per_id':num_images,
                                                'total_training_images':len(pd.read_csv(train_file)),
                                                'batch_size':config['data_settings']['batch_size'],
                                                'num_epochs':config['train_settings']['num_epochs'],
                                                'train_loss':results['train_loss'],
                                                'valid_loss':results['valid_loss'],
                                                '1NN':results['1NN_acc'],
                                                '3NN':results['3NN_acc'],
                                                'training_file':train_file,
                                                'reference_file':reference_file,
                                                'query_file':query_file,
                                                'start_time':results['start_time'],
                                                'train_time':results['train_time'],
                                                'stop_epoch':results['stop_epoch']}
            results_df.to_csv(config['eval_settings']['results_file'],index=False)




    



/home/gsantiago/summer_bee_data/open_sets/new_open_04_ids_all_colors_batch1/summer_bee_dataset_open_train_bee_4_ids_batch1_sample_num_max.csv
/home/gsantiago/summer_bee_data/open_sets/new_open_04_ids_all_colors_batch1/summer_bee_dataset_open_train_bee_4_ids_batch1_sample_num_16.csv
/home/gsantiago/summer_bee_data/open_sets/new_open_04_ids_all_colors_batch1/summer_bee_dataset_open_train_bee_4_ids_batch1_sample_num_04.csv
32 4


FileNotFoundError: [Errno 2] No such file or directory: './reid_template.yml'

#### In event of run failure to record automatically
Use below code to save run details from results.pickle even if there was an issue in your run

In [5]:

with open(config['eval_settings']['pickle_file'],'rb') as fi:
    results = pickle.load(fi)   

# Write out run summary to results tracking document
results_df = pd.read_csv(config['eval_settings']['results_file'])
results_df.loc[len(results_df)] = {'run_str': run_str,
                                    'wandb_id':results['wandb_id'],
                                    'num_ids':num_ids,
                                    'num_images_per_id':num_images,
                                    'total_training_images':len(pd.read_csv(train_file)),
                                    'batch_size':config['data_settings']['batch_size'],
                                    'num_epochs':config['train_settings']['num_epochs'],
                                    'train_loss':results['train_loss'],
                                    'valid_loss':results['valid_loss'],
                                    '1NN':results['1NN_acc'],
                                    '3NN':results['3NN_acc'],
                                    'training_file':train_file,
                                    'reference_file':reference_file,
                                    'query_file':query_file,
                                    'start_time':results['start_time'],
                                    'train_time':results['train_time'],
                                    'stop_epoch':results['stop_epoch']}
results_df.to_csv(config['eval_settings']['results_file'],index=False)

